<div class='status' style='background-color: #0c4a6e; color: white; padding-top: 4px; padding-bottom: 4px; padding-left: 20px; padding-right: 20px; border-radius: 10px; font-family: Arial, sans-serif; font-size: 26px; display: inline-block; text-align: center; box-shadow: 0px 3px 4px rgba(0, 0, 0, 0.5);'><b>Notion Agents</b> - Calendar Scheduling <code>01</code></div>

1. **Notion Database Data Loader** - Custom Crewai Tool to load all information form a Notion Database including a list of page ids.
2. **Notion Page Data Loader** - Tool to load all the json information from a Notion database page, including creator information and comments.


<div class='status' style='background-color: #f59e0b; color: white; padding-top: 2px; padding-bottom: 2px; padding-left: 7px; padding-right: 7px; border-radius: 6px; font-family: Arial, sans-serif; font-size: 18px; display: inline-block; text-align: center; box-shadow: 0px 3px 4px rgba(0, 0, 0, 0.2);'><b>Loading</b> Libraries</div>

In [166]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import json
import yaml
from crewai import Agent, Task, Crew

from datetime import datetime
today = datetime.now()
print(today)

%load_ext autoreload
%autoreload 2

2024-12-17 11:56:15.378776
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [167]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'

<div class='status' style='background-color: #0e7490; color: white; padding-top: 2px; padding-bottom: 2px; padding-left: 7px; padding-right: 7px; border-radius: 6px; font-family: Arial, sans-serif; font-size: 18px; display: inline-block; text-align: center; box-shadow: 0px 3px 4px rgba(0, 0, 0, 0.2);'>Loading <b>Agents + Tasks</b> form YAML file</div>

In [168]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

<div class='status' style='background-color: #0e7490; color: white; padding-top: 2px; padding-bottom: 2px; padding-left: 7px; padding-right: 7px; border-radius: 6px; font-family: Arial, sans-serif; font-size: 18px; display: inline-block; text-align: center; box-shadow: 0px 3px 4px rgba(0, 0, 0, 0.2);'>Create <b>Custom Tools</b></div>

**Specify Notion Database ID** for report.  
Moving forward this should be passed to the crew as an input paramter.

Appraisals Database:  12dfdfd68a97808889d4d8c041a281eb

In [169]:
from crewai_tools import BaseTool
from notion_api.notionhelper import *
import requests
from typing import ClassVar, Union, Dict, Any
from dotenv import load_dotenv
import requests

NOTION_TOKEN = os.getenv('NOTION_TOKEN')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
NOTION_ENDPOINT = os.getenv('NOTION_ENDPOINT')
NOTION_DATABASE_ID = os.getenv('NOTION_DATABASE_ID')

# Validate if the variables are loaded correctly
if not NOTION_TOKEN or not OPENAI_API_KEY or not NOTION_ENDPOINT or not NOTION_DATABASE_ID:
    raise ValueError("One or more required environment variables are missing.")

class DatabaseDataFetcherTool(BaseTool):
    name: str = "Notion Database Data Fetcher"
    description: str = "Fetches Notion Database structure and properties."

    nh: ClassVar[NotionHelper] = NotionHelper()
    database_id: str = "154fdfd68a978048b819ecdf57801205"
    
    def _run(self) -> Union[dict, str]:
        """
        Fetch all the properties and structure of a Notion database.
        """
        response = self.nh.get_database(database_id=self.database_id)
        return response
        
        if isinstance(response, dict):
            return response
        else:
            # Fallback in case of timeouts or other issues
            return {"error": "Failed to fetch data from Notion. Please try again."}


class PageDataFetcherTool(BaseTool):
    """
    Tool to fetch all pages in a Notion database with their IDs and full schemas.
    """
    name: str = "Fetch Notion Pages Tool"
    description: str = (
        "Fetches all pages in a specified Notion database, "
        "returning their unique page IDs and full page schemas."
    )

    headers: ClassVar[Dict[str, str]] = {
        "Authorization": f"Bearer {NOTION_TOKEN}",
        "Content-Type": "application/json",
        "Notion-Version": NOTION_VERSION
    }

    def _run(self, database_id: str) -> List[Dict[str, Any]]:
        """
        Fetches all pages in the specified Notion database.

        Args:
            database_id (str): The Notion database ID.

        Returns:
            List[Dict]: A list of dictionaries containing page IDs and full schemas.
        """
        url = f"{NOTION_ENDPOINT}/databases/{database_id}/query"
        all_pages = []
        has_more = True
        next_cursor = None

        # Handle pagination to get all pages
        while has_more:
            payload = {"start_cursor": next_cursor} if next_cursor else {}
            response = requests.post(url, headers=self.headers, json=payload)

            if response.status_code == 200:
                data = response.json()
                for page in data.get("results", []):
                    all_pages.append({
                        "page_id": page.get("id"),
                        "full_schema": page
                    })

                has_more = data.get("has_more", False)
                next_cursor = data.get("next_cursor", None)
            else:
                return [{"error": response.text, "status_code": response.status_code}]

        return all_pages


class NewTaskCreationTool(BaseTool):
    name: str = "Create New Task Tool"
    description: str = "Creates a new task in the calendar database with user-specified properties like Priority, Title, and Due Dates."

    headers: ClassVar[Dict[str, str]] = {
        "Authorization": f"Bearer {NOTION_TOKEN}",
        "Content-Type": "application/json",
        "Notion-Version": "2022-06-28"  # Use the correct Notion API version
    }

    def _run(self, name: str, status: str, priority: str, start_datetime: str, end_datetime: str) -> Dict[str, Any]:
        """
        Create a new task in the Notion database with specified properties.

        Args:
            title (str): The name of the task.
            status (str): The status of the task (e.g., "Not Started", "In progress", "Done").
            priority (str): Priority of the task ("High", "Medium", "Low").
            start_datetime (str): Start date and time in ISO 8601 format.
            end_datetime (str): End date and time in ISO 8601 format.

        Returns:
            dict: The response from the Notion API.
        """
        task_data = {
            "parent": {"database_id": NOTION_DATABASE_ID},
            "properties": {
                "Name": {
                    "title": [{"text": {"content": name}}]
                },
                "Status": {
                    "status": {"name": status}
                },
                "Priority": {
                    "select": {"name": priority}
                },
                "Due Date": {
                    "date": {"start": start_datetime, "end": end_datetime}
                }
            }
        }

        # Make the API request to Notion
        url = f"{NOTION_ENDPOINT}/pages"
        response = requests.post(url, headers=self.headers, json=task_data)

        if response.status_code == 200:
            return response.json()
        else:
            return {"error": response.text}


class RescheduleExcistingTasks(BaseTool):
    name: str = "Reschedule Existing Task Tool"
    description: str = "Reschedules an existing task identified by page_id, with a new start date time and end date time."

    headers: ClassVar[Dict[str, str]] = {
        "Authorization": f"Bearer {NOTION_TOKEN}",
        "Content-Type": "application/json",
        "Notion-Version": "2022-06-28"  # Use the correct Notion API version
    }

    def _run(self, page_id: str, start_datetime: str, end_datetime: str) -> Dict[str, Any]:
        """
        Update the start_datetime and end_datetime properties for an excisting Task or database page.

        Args:
            page_id (str): Notioh Page ID of page to be updated or rescheduled.
            start_datetime (str): Start date and time in ISO 8601 format.
            end_datetime (str): End date and time in ISO 8601 format.

        Returns:
            dict: The response from the Notion API.
        """
        task_data = {
            "parent": {"database_id": NOTION_DATABASE_ID},
            "properties": {
                "Due Date": {
                    "date": {"start": start_datetime, "end": end_datetime}
                }
            }
        }

        # Make the API request to Notion
        url = f"{NOTION_ENDPOINT}/pages/{page_id}"
        response = requests.patch(url, headers=self.headers, json=task_data)

        if response.status_code == 200:
            return response.json()
        else:
            return {"error": response.text}

<div class='status' style='background-color: #0e7490; color: white; padding-top: 2px; padding-bottom: 2px; padding-left: 7px; padding-right: 7px; border-radius: 6px; font-family: Arial, sans-serif; font-size: 18px; display: inline-block; text-align: center; box-shadow: 0px 3px 4px rgba(0, 0, 0, 0.2);'>Create <b>Crew, Agents and Tasks</b></div>

In [170]:
from crewai import LLM
   

# Creating Agents

data_collection_agent = Agent(
  config=agents_config['data_collection_agent'],
  tools=[DatabaseDataFetcherTool(), PageDataFetcherTool()],

)

calendar_scheduler_agent = Agent(
  config=agents_config['calendar_scheduler_agent'],
)

data_update_agent = Agent(
  config=agents_config['data_update_agent'],
  tools=[NewTaskCreationTool()],
)

# Creating Tasks
data_collection = Task(
  config=tasks_config['data_collection'],
  agent=data_collection_agent
)


schedule_tasks = Task(
  config=tasks_config['schedule_tasks'],
  agent=calendar_scheduler_agent,
  tools=[RescheduleExcistingTasks()],
)

update_database_properties = Task(
  config=tasks_config['update_database_properties'],
  agent=data_update_agent,
  tools=[RescheduleExcistingTasks()],
)

# Creating Crew
crew = Crew(
  agents=[
    data_collection_agent,
    calendar_scheduler_agent,
    data_update_agent
  ],
  tasks=[
    data_collection,
    schedule_tasks,
    update_database_properties 
  ],
  verbose=True
)

2024-12-17 11:56:15,557 - 8316356672 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


<div class='status' style='background-color: #4d7c0f; color: white; padding-top: 2px; padding-bottom: 2px; padding-left: 7px; padding-right: 7px; border-radius: 6px; font-family: Arial, sans-serif; font-size: 18px; display: inline-block; text-align: center; box-shadow: 0px 3px 4px rgba(0, 0, 0, 0.2);'><b>Train</b> the Crew</div>

In [171]:
#crew.train(n_iterations=1, filename='training.pkl')

In [172]:
#crew.test(3)

<div class='status' style='background-color: #be123c; color: white; padding-top: 2px; padding-bottom: 2px; padding-left: 7px; padding-right: 7px; border-radius: 6px; font-family: Arial, sans-serif; font-size: 18px; display: inline-block; text-align: center; box-shadow: 0px 3px 4px rgba(0, 0, 0, 0.2);'><b>Kickoff</b> the Crew</div>

In [173]:
prompt = 'I have a new task to Phone HMRC re PAYE today at 10:00 expected to take 30 min, create this task and reschedule my other commitments accordingly. \
        Move any tasks not completed yesterday to today for completion. to reschedule tasks use the RescheduleExcistingTasks Tool'
datetime_now = today

In [174]:
# The given Python dictionary
inputs = {
  'prompt': prompt,
  'datetime_now': datetime_now
}

# Run the crew
result = crew.kickoff(
  inputs=inputs
)

# Agent: Data Collection Specialist
## Task: Create an initial understanding of the notion database by reviewing the database schema. Use the Notion Databse Data Fetcher tool to gather data from the Notion database, this explains the structure of the database. Then use the PageDataFetcher Tool to retrieve all page data with corresponding page_id's for each page / task.



# Agent: Data Collection Specialist
## Thought: I need to start by gathering the structure and properties of the Notion database to understand its schema thoroughly.
## Using tool: Notion Database Data Fetcher
## Tool Input: 
"{}"
## Tool Output: 
{'object': 'database', 'id': '154fdfd6-8a97-8048-b819-ecdf57801205', 'cover': None, 'icon': None, 'created_time': '2024-12-06T20:35:00.000Z', 'created_by': {'object': 'user', 'id': '25a73f97-6d2f-4d6f-99ac-ec5ae2454f5c'}, 'last_edited_by': {'object': 'user', 'id': '25a73f97-6d2f-4d6f-99ac-ec5ae2454f5c'}, 'last_edited_time': '2024-12-17T10:54:00.000Z', 'title': [{'type': 'tex

### **Cost** Estimation

In [175]:
import pandas as pd

costs = 0.150 * (crew.usage_metrics.prompt_tokens + crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])
df_usage_metrics

Total costs: $0.0037


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,24667,21671,10112,2996,8


<div class='status' style='background-color: #0f766e; color: white; padding-top: 2px; padding-bottom: 2px; padding-left: 7px; padding-right: 7px; border-radius: 6px; font-family: Arial, sans-serif; font-size: 18px; display: inline-block; text-align: center; box-shadow: 0px 3px 4px rgba(0, 0, 0, 0.2);'>Crewai<b> Final Output</b></div>

In [176]:
from IPython.display import Markdown

markdown  = result.raw
Markdown(markdown)

No changes or new tasks required at the moment. All tasks are scheduled correctly.